### OpenAI Functions using Gorilla LLM model
This example is to help understand and experiment OpenFunction
Model used here  is gorilla-openfunctions-v0

Ref - https://gorilla.cs.berkeley.edu/blogs/4_open_functions.html

In [5]:
import json
import openai

# Example dummy function hard coded to return the same weather for now
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "46 "+ unit,
        "unit": unit,
        "forecast": ["rainy", "windy"],
    }
    return json.dumps(weather_info)

In [6]:
# define a function.
# query can be something like, Get current weather in seattle, WA
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. Seattle, WA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [7]:
def get_response_from_model(messages, function_name, model = "gorilla-openfunctions-v1"):
    openai.api_key = "EMPTY"
    openai.api_base =  "http://luigi.millennium.berkeley.edu:8000/v1"

    response = openai.ChatCompletion.create(
                model = model,
                messages = messages,
                functions = functions,
                function_call = {"name":function_name}
            )
    return response

In [8]:
messages = [{
        "role": "user",
        "content": "What's the weather like in Seattle?"
    }]

get_response_from_model(messages=messages,function_name="get_current_weather" )

#### LangChain Expression Language (LCEL)

In [20]:
# Generate function doc using pydantic 
from pydantic import BaseModel, Field
from enum import Enum

class units(str, Enum):
    pear = 'fahrenheit'
    banana = 'celsius'

class WeatherReport(BaseModel):
    """Get the current weather in a given location"""
    location: str = Field(description="The city and state, e.g. Seattle, WA")
    unit: units = Field(description="enum celsius or fahrenheit")

In [21]:
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
import json
weather_report_function = convert_pydantic_to_openai_function(WeatherReport)
weather_report_function

{'name': 'WeatherReport',
 'description': 'Get the current weather in a given location',
 'parameters': {'$defs': {'units': {'enum': ['fahrenheit', 'celsius'],
    'title': 'units',
    'type': 'string'}},
  'description': 'Get the current weather in a given location',
  'properties': {'location': {'description': 'The city and state, e.g. Seattle, WA',
    'title': 'Location',
    'type': 'string'},
   'unit': {'allOf': [{'enum': ['fahrenheit', 'celsius'],
      'title': 'units',
      'type': 'string'}],
    'description': 'enum celsius or fahrenheit'}},
  'required': ['location', 'unit'],
  'title': 'WeatherReport',
  'type': 'object'}}